In [ ]:
from thesis_initialise import *

In [ ]:
from collections.abc import Mapping as _Mapping
import numbers as _numbers
import functools as _functools
import inspect as _inspect
import weakref as _weakref

from everest.h5anchor import disk as _disk
from everest import reseed as _reseed
from everest.funcy import (
    Function as _Function,
    generic as _generic,
    special as _special,
    Fn as _Fn,
    )
from everest.cascade import (
    Cascade as _Cascade,
    Inputs as _Inputs,
    )
from everest.funcy.base import Base as _Base
from everest import wordhash as _wordhash
from everest.exceptions import *

In [ ]:
################################################################################

class SchemaIncisable(_generic.FuncyHardIncisable):
    def __init__(self, schema):
        self._schema = schema
    @property
    def schema(self):
        return self._schema
    @property
    def shape(self):
        return self.schema.shape

################################################################################

class Schema(type):
    userdefined = False
    def __new__(meta, name, bases, dic, *args, **kwargs):
        cls = super().__new__(meta, name, bases, dic)
        cls.root = cls
        if cls.userdefined:
            cls.script = script = \
                str(_reseed.digits(12)) # <- TEMPORARY
    #             _disk.ToOpen(inspect.getfile(schema))()
            cls._schemaID = schemaID = \
                _wordhash.get_random_proper(2, seed = script)
            try:
                cls = meta._premade[schemaID]
                assert cls.script == script, (script[:32], schema.script[:32])
                raise KeyError
            except KeyError:
                meta._premade[schemaID] = cls
        else:
            cls._schemaID = f'EverestSchema({cls.__name__})'
            return cls
        return cls
    @property
    def defaults(self):
        try:
            return self._defaults
        except AttributeError:
            defaults = self._defaults = _Inputs(
                self.__init__,
                name = self.__name__
                )
            return defaults
    @property
    def inputs(self):
        return self.defaults
    @property
    def schemaID(self):
        return self.root._schemaID
    @property
    def hashID(self):
        return self.schemaID
    def __call__(self, *args, **kwargs):
        instance = object.__new__(self)
        inputs = self.inputs.copy(*args, **kwargs)
        setArgs, setKwargs = inputs.args, inputs.kwargs
        instance.__init__(*setArgs, **{**kwargs, **setKwargs})
        instanceID = str(_reseed.digits(12))
        instance.instanceID = instanceID
        instance.hashID = self.hashID + ';' + instanceID
        instance.inputs = inputs
        return instance

    @property
    def schemaIncisable(self):
        try:
            return self._schemaIncisable(self)
        except AttributeError:
            self._schemaIncisable = schInc = SchemaIncisable(self)
            return schInc
    @property
    def shape(self) -> tuple:
        return (_special.infint,)
    def __getitem__(self, arg, /):
        return self.schemaIncisable[arg]

def make_case(cls, *args, **kwargs):
    if not isinstance(cls, Schema):
        raise TypeError
    case = _Cascade(*args, **kwargs)
    case = Case(
        f"{cls.__name__}:{case.hashID}",
        (cls, *cls.__bases__),
        dict(cls.__dict__),
        case = case,
        )
    return case

class Case(Schema):
    def __init__(self, *args, case: _Cascade, **kwargs):
        self._case = case
        super().__init__(*args, **kwargs)
    @property
    def inputs(self):
        return self.defaults.copy(**self._case)

################################################################################

class Basic(metaclass = Schema):
    def __repr__(self):
        return self.hashID

################################################################################

class MyClass(Basic):
    userdefined = False
    def __init__(self,
            a = 1,
            b = 2,
            c = 3,
            _d = 4,
            **kwargs,
            ):
        self.foo = a * b * c
        super().__init__(**kwargs)

In [ ]:
MyClass.hashID

In [ ]:
case = make_case(MyClass, a = 10)
case()

In [ ]:
case.hashID

In [ ]:
case.inputs

In [ ]:
case().inputs

In [ ]:
case.inputs

In [ ]:
myinst.inputs

In [ ]:
case.defaults

In [ ]:
case.inputs

In [ ]:
MyClass

In [ ]:
MyClass[dict(a = 10)]

In [ ]:
mystrict = SchemaIncisable(MyClass)[dict(a = 1)]

In [ ]:
mystrict.source

In [ ]:
mystrict.incisor

In [ ]:
myinst = MyClass(a = 10)

In [ ]:
MyClass.schemaIncisable

In [ ]:
myobj = mycase()

In [ ]:
type(myobj).shape

In [ ]:
repr(myobj)

In [ ]:
myobj

In [ ]:
################################################################################

class Schema(_generic.FuncyHardIncisable, type):
    userdefined = False
    def __new__(meta, name, bases, dic, *args, **kwargs):
        cls = super().__new__(meta, name, bases, dic)
        cls.root = cls
        if cls.userdefined:
            cls.script = script = \
                str(_reseed.digits(12)) # <- TEMPORARY
    #             _disk.ToOpen(inspect.getfile(schema))()
            cls._schemaID = schemaID = \
                _wordhash.get_random_proper(2, seed = script)
            try:
                cls = meta._premade[schemaID]
                assert cls.script == script, (script[:32], schema.script[:32])
                raise KeyError
            except KeyError:
                meta._premade[schemaID] = cls
        else:
            cls._schemaID = f'EverestSchema_{cls.__name__}'
            return cls
        return cls
    def __init__(self, *args, defaults = None, **kwargs):
        if not defaults is None:
            self._defaults = defaults
        self.cases = _weakref.WeakValueDictionary()
        super().__init__(*args, **kwargs)
    @property
    def incisionTypes(self):
        return {**super().incisionTypes, **dict(declarative = Case)}
    @property
    def defaults(self):
        try:
            return self._defaults
        except AttributeError:
            defaults = self._defaults = _Inputs(
                self.__init__,
                name = self.__name__
                )
            return defaults
    @property
    def schemaID(self):
        return self.root._schemaID
    @property
    def hashID(self):
        return self.schemaID
    @classmethod
    def _get_incision_method(cls, arg):
        if isinstance(arg, _Function):
            if arg.isSeq:
                return cls._getitem_seq
        return super()._get_incision_method(arg)
    def _getitem_declarative(self, arg):
        return self._get_case(arg)
    def _get_case(self, *args, **kwargs):
        case = _Cascade(*args, **kwargs)
        try:
            case = self.cases[(hashID := case.hashID)]
        except KeyError:
            case = self._get_incision_type('declarative')(
                f"{self.root.__name__}:{case.hashID}",
                (self, *self.__bases__),
                dict(self.__dict__),
                case = case,
                defaults = self.defaults,
                )
            case.root = self.root
            self.cases[hashID] = case
        return case
    def __call__(self, *args, **kwargs):
        return self._get_case(*args, **kwargs)(**kwargs)
    @property
    def shape(self) -> tuple:
        return (_special.infint,)

class Case(_generic.FuncyStrictIncision, Schema):
#     def __init__(self, *args, case: _Cascade, **kwargs):
#         self._case = case
#         self._inputs = self.defaults.copy(**case)
#         super().__init__(*args, **kwargs)
    @property
    def inputs(self):
        return self._inputs
    @property
    def case(self):
        return self._case
    @property
    def hashID(self):
        return self.schemaID + ':' + self.case.hashID
    @property
    def shape(self):
        _, *dims = super().shape
        return tuple(dims)
    def __call__(self, **kwargs):
        instance = object.__new__(self)
        inputs = self.inputs
        setArgs, setKwargs = inputs.args, inputs.kwargs
        instance.__init__(*setArgs, **{**kwargs, **setKwargs})
        instance.case = self.case
        instanceID = str(_reseed.digits(12))
        instance.instanceID = instanceID
        instance.hashID = self.hashID + ';' + instanceID
        instance.inputs = inputs
        return instance

################################################################################

class Basic(metaclass = Schema):
    ...

################################################################################

class MyClass(Basic):
    userdefined = False
    def __init__(self,
            a = 1,
            b = 2,
            c = 3,
            _d = 4,
            **kwargs,
            ):
        self.foo = a * b * c
        super().__init__(**kwargs)